In [1]:

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

BASE = "https://books.toscrape.com/"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

books = []


url = BASE
while True:
    print("Fetching:", url)
    resp = requests.get(url, headers=headers, timeout=15)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "lxml")


    for article in soup.select("article.product_pod"):

        a = article.select_one("h3 a")
        title = a["title"].strip()


        price_tag = article.select_one("p.price_color")
        price = price_tag.text.strip() if price_tag else ""


        avail_tag = article.select_one("p.instock.availability")
        availability = avail_tag.text.strip() if avail_tag else ""


        star_tag = article.select_one("p.star-rating")
        star_rating = ""
        if star_tag:
            classes = star_tag.get("class", [])

            for c in classes:
                if c.lower() not in ("star-rating",):
                    star_rating = c
                    break

        books.append({
            "Title": title,
            "Price": price,
            "Availability": availability,
            "Star Rating": star_rating
        })


    next_li = soup.select_one("li.next a")
    if next_li:
        next_href = next_li.get("href")

        url = requests.compat.urljoin(url, next_href)
        time.sleep(1)
    else:
        break


df = pd.DataFrame(books)
df.to_csv("books.csv", index=False, encoding="utf-8")
print("Saved", len(df), "books to books.csv")


Fetching: https://books.toscrape.com/
Fetching: https://books.toscrape.com/catalogue/page-2.html
Fetching: https://books.toscrape.com/catalogue/page-3.html
Fetching: https://books.toscrape.com/catalogue/page-4.html
Fetching: https://books.toscrape.com/catalogue/page-5.html
Fetching: https://books.toscrape.com/catalogue/page-6.html
Fetching: https://books.toscrape.com/catalogue/page-7.html
Fetching: https://books.toscrape.com/catalogue/page-8.html
Fetching: https://books.toscrape.com/catalogue/page-9.html
Fetching: https://books.toscrape.com/catalogue/page-10.html
Fetching: https://books.toscrape.com/catalogue/page-11.html
Fetching: https://books.toscrape.com/catalogue/page-12.html
Fetching: https://books.toscrape.com/catalogue/page-13.html
Fetching: https://books.toscrape.com/catalogue/page-14.html
Fetching: https://books.toscrape.com/catalogue/page-15.html
Fetching: https://books.toscrape.com/catalogue/page-16.html
Fetching: https://books.toscrape.com/catalogue/page-17.html
Fetching: 

In [3]:

!pip install requests beautifulsoup4 pandas lxml selenium webdriver-manager

!apt-get update -y
!apt-get install -y chromium-browser chromium-chromedriver




Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://cli.github.com/packages stable InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:11 http://security.ubuntu.com/ubuntu jammy-security InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

In [5]:
!apt-get update -y
!apt-get install -y wget unzip
!wget -q https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!apt-get install -y ./google-chrome-stable_current_amd64.deb

!pip install selenium webdriver-manager


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:5 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:8 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

url = "https://www.imdb.com/chart/top/"
driver.get(url)
time.sleep(2)

rows = driver.find_elements(By.CSS_SELECTOR, "table.chart.full-width tbody tr")

movies = []
for row in rows:
    title_col = row.find_element(By.CSS_SELECTOR, "td.titleColumn")
    rank = int(title_col.text.split('.')[0].strip())
    title = title_col.find_element(By.TAG_NAME, "a").text.strip()
    year = title_col.find_element(By.CLASS_NAME, "secondaryInfo").text.strip("() ")
    rating = row.find_element(By.CSS_SELECTOR, "td.ratingColumn.imdbRating strong").text.strip()

    movies.append({
        "Rank": rank,
        "Movie Title": title,
        "Year of Release": year,
        "IMDB Rating": rating
    })

df_imdb = pd.DataFrame(movies)
df_imdb.to_csv("imdb_top250.csv", index=False, encoding="utf-8")
print("saved to imdb_top250.csv")

driver.quit()


saved to imdb_top250.csv


In [4]:

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from urllib.parse import urljoin

BASE = "https://www.timeanddate.com/weather/"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

resp = requests.get(BASE, headers=headers, timeout=15)
resp.raise_for_status()
soup = BeautifulSoup(resp.text, "lxml")

cities = []


tables = soup.select("table.zebra, table.tb-wt, table.table--left")

rows = []
for t in tables:
    rows.extend(t.select("tbody tr"))


if not rows:

    anchors = [a for a in soup.select("a") if a.get("href", "").startswith("/weather/")]

    anchors = [a for a in anchors if len(a.text.strip()) > 0 and len(a.text.strip()) < 40]

    for a in anchors:
        parent = a.parent

        temp = None
        cond = None

        for sibling in parent.find_all(string=True, recursive=False):
            s = sibling.strip()
            if s and ("°C" in s or "°F" in s):
                temp = s
                break

        img = parent.find("img")
        if img and img.get("alt"):
            cond = img.get("alt").strip()
        rows.append((a, temp, cond))
else:

    for tr in rows:

        a = tr.find("a")
        if not a:
            continue
        city_name = a.text.strip()

        temp = ""
        cond = ""

        temp_td = tr.find("td", class_="r") or tr.find_all("td")[-1] if tr.find_all("td") else None
        if temp_td:
            temp = temp_td.text.strip()

        img = tr.find("img")
        if img and img.get("alt"):
            cond = img.get("alt").strip()

        cities.append({
            "City Name": city_name,
            "Temperature": temp,
            "Weather Condition": cond
        })


if rows and isinstance(rows[0], tuple):
    cities = []
    for a, temp, cond in rows:
        cities.append({
            "City Name": a.text.strip(),
            "Temperature": temp or "",
            "Weather Condition": cond or ""
        })


if not cities:
    print("No city data parsed. Page layout may differ. Try inspecting the page or run with network available.")
else:
    df = pd.DataFrame(cities)
    df.to_csv("weather.csv", index=False, encoding="utf-8")
    print("Saved", len(df), "cities to weather.csv")


Saved 158 cities to weather.csv
